In [40]:
import getpass
import pyodbc
import pandas as pd
from termcolor import colored, cprint
from datetime import datetime

In [41]:
def get_sql_connection():
    
    try:
        # connect to SQL Server.
        SERVER = '34.206.73.189' 
        DATABASE = 'datascience' 
        USERNAME = 'nrad' 
        PASSWORD = '83F25619-D272-4660-98A2-93AF5CC18D59' #getpass.getpass(prompt='Enter your password')
        CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
        
        print("The Connection to SQL was succesfull")
    
    except Exception as err:
            print("The Connection to SQL wasn't succesfull", err)
    
    return CNXN
            

In [42]:
def get_datetime():
    
    today = datetime.now()
    
    todaydate = today.strftime("%Y-%m-%d")
    todaydate_id = today.strftime("%Y%m%d")
    
    return todaydate, todaydate_id

In [50]:
def get_teams(conn):
    
    cursor = conn.cursor()
    
    sql =  f"""
        SELECT DISTINCT
            clientcode,
            lkupclientid,
            stlrDBName,
            leagueName
        FROM 
            ds.productyear_all
        """
    
    df_teams = pd.read_sql(sql, conn)
    
    return df_teams

In [59]:
def get_ds_scores(conn,lkupclientid_value,clientcode_value,todaydate):
    try:

        cursor = conn.cursor()
        numRecords =  f'''select count(*) as numRecords, max(cast(insertDate as date)) as maxInserDate  from ds.customerScores where lkupclientid ={lkupclientid_value} ;'''
        cntTodayScores = pd.read_sql(numRecords, conn)
        cntrecords= cntTodayScores['numRecords']
        insertdate = cntTodayScores['maxInserDate']
        
        for idx, m in enumerate(cntrecords):
            updatedScores = pd.DataFrame(columns=['clientCode_Updated', 'lkupClientId', 'NumRecords','InsertDate'])
            stalesScores = pd.DataFrame(columns=['clientCode_NotUpdated', 'lkupClientId', 'NumRecords','InsertDate'])
            value=cntrecords[idx]
            insertdate_idx = insertdate[idx]
            if str(insertdate_idx) == str(todaydate):
                updatedScores = updatedScores.append({'clientCode_updated': clientcode_value, 'lkupClientId': lkupclientid_value, 'NumRecords': value, 'InsertDate': insertdate_idx}, ignore_index=True)
                #print(str(clientcode_value)+ "-"+str(lkupclientid_value) + ":"+ " is updated in customerScores table and " + str(value) + " values has been inserted. "  )
            elif str(insertdate_idx) != str(todaydate):
                stalesScores = stalesScores.append({'clientCode_NotUpdated': clientcode_value, 'lkupClientId': lkupclientid_value, 'NumRecords': 0, 'InsertDate': insertdate_idx}, ignore_index=True)
                #print(colored(str(clientcode_value)+ "-" +str(lkupclientid_value) + ":"+ " is not updated in customerScores table. the last updateDate is: " + str(insertdate_idx), 'red', attrs=['reverse', 'blink']))
                
        conn.commit()
        cursor.close()
        
        
    except Exception as err:
            print("The Connection wasn't succesfull", err)
    print(updatedScores.head())
            
    return updatedScores,stalesScores




In [ ]:
def get_ds_scores(conn,lkupclientid_value,clientcode_value,todaydate):
    try:

        cursor = conn.cursor()
        numRecords =  f'''select count(*) as numRecords, max(cast(insertDate as date)) as maxInserDate  from ds.customerScores where lkupclientid ={lkupclientid_value} ;'''
        cntTodayScores = pd.read_sql(numRecords, conn)
        cntrecords= cntTodayScores['numRecords']
        insertdate = cntTodayScores['maxInserDate']
        
        for idx, m in enumerate(cntrecords):
            value=cntrecords[idx]
            insertdate_idx = insertdate[idx]
            if str(insertdate_idx) == str(todaydate):
                print(str(clientcode_value)+ "-"+str(lkupclientid_value) + ":"+ " is updated in customerScores table and " + str(value) + " values has been inserted. "  )
            elif str(insertdate_idx) != str(todaydate):
                print(colored(str(clientcode_value)+ "-" +str(lkupclientid_value) + ":"+ " is not updated in customerScores table. the last updateDate is: " + str(insertdate_idx), 'red', attrs=['reverse', 'blink']))
                
        conn.commit()
        cursor.close()
        
    except Exception as err:
            print("The Connection wasn't succesfull", err)




In [60]:
def get_stlr_scores(lkupclientid_value,clientcode_value,stlrDBName_value,todaydate_id):
    
    
    try:
        SERVER = '34.206.73.189'
        DATABASE = stlrDBName_value 
        USERNAME = 'nrad' 
        PASSWORD = '83F25619-D272-4660-98A2-93AF5CC18D59' #getpass.getpass(prompt='Enter your password')
        CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
        cursor = CNXN.cursor()
        numRecords_fact =  f'''select count(*) as factnumRecords,max(dimdateid) as maxInserDate from dw.factCustomerretentionScore where lkupclientid ={lkupclientid_value} ;'''
        cntTodayScores_fact = pd.read_sql(numRecords_fact, CNXN)
        cntrecords_fact = cntTodayScores_fact['factnumRecords']
        insertdate_fact = cntTodayScores_fact['maxInserDate'] 
        for idx, m in enumerate(cntrecords_fact):
            value=cntrecords_fact[idx]
            insertdatefact_idx = insertdate_fact[idx]
            if str(insertdatefact_idx) == str(todaydate_id):
                print(str(clientcode_value)+ "-"+str(lkupclientid_value) + ":"+ " is updated and " + str(value) + " values has been inserted in factCustomerretentionScore table. "  )
            elif str(insertdatefact_idx) != str(todaydate_id):
                print(colored(str(clientcode_value)+ "-" +str(lkupclientid_value) + ":"+ " is not updated in factCustomerretentionScore table. the last updateDate is: " + str(insertdatefact_idx), 'red', attrs=['reverse', 'blink']))
                
        CNXN.commit()
        cursor.close()
    
    except Exception as err:
        print("The Connection to wasn't succesfull to " + stlrDBName_value, err)

In [61]:
if __name__ == "__main__":
    
    # get the current date and date_id
    todaydate, todaydate_id = get_datetime()
    
    # get a connection to sql
    conn = get_sql_connection()
    
    # get a list of all teams
    df_teams = get_teams(conn)

    # loop through all teams
    for index, row in df_teams.iterrows():
        lkupclientid_value = row['lkupclientid']
        clientcode_value = row['clientcode'] 
        stlrDBName_value = row['stlrDBName']
        # call get_ds_scores()
        #stalesScores_new,stalesScores_new = get_ds_scores(conn,lkupclientid_value,clientcode_value,todaydate)
        get_ds_scores(conn,lkupclientid_value,clientcode_value,todaydate)
    #print(stalesScores_new)
        
        #get_stlr_scores(lkupclientid_value,clientcode_value,stlrDBName_value,todaydate_id)
        
        #print(row['lkupclientid'], row['clientcode'])
    
    # loop through all teams
    #for idx, m in enumerate(df_teams):
        #lkupclientid_value=lkupclientid_df[idx]
        #clientcode_value=clientcode_df[idx]
        #print(str(lkupclientid_value)+"-"+str(clientcode_value))
        #get_ds_scores(conn,lkupclientid_value,clientcode_value)
        
        # call get_ds_scores()
            # return a dataframe of teams with updated scores in DS db
            # return a dataframe of teams with stales scores in DS db
        
        # call get_stlr_scores()
            # return a dataframe of teams with updated scores in STLR db
            # return a dataframe of teams with stales scores in STRL db
        
    # print a report of teams that have been updated and how many records
    
    # print a report of teams that have NOT been updated and how many records
    

    #conn = get_db_connection("PRD")
    #retention_scores = get_ds_scores(conn)
    #fact_retention_scores = get_stlr_scores(conn)




The Connection to SQL was succesfull
  clientCode_Updated lkupClientId NumRecords  InsertDate clientCode_updated
0                NaN           19      24434  2022-03-14              66ers
  clientCode_Updated lkupClientId NumRecords  InsertDate clientCode_updated
0                NaN           16     119466  2022-03-14           admirals
  clientCode_Updated lkupClientId NumRecords  InsertDate clientCode_updated
0                NaN            5     485886  2022-03-14            blazers
  clientCode_Updated lkupClientId NumRecords  InsertDate clientCode_updated
0                NaN           11     310395  2022-03-14              bulls
  clientCode_Updated lkupClientId NumRecords  InsertDate clientCode_updated
0                NaN            7     431906  2022-03-14            canucks
  clientCode_Updated lkupClientId NumRecords  InsertDate clientCode_updated
0                NaN           55     183761  2022-03-14            coyotes
  clientCode_Updated lkupClientId NumRecords  Inser